<a href="https://colab.research.google.com/github/rahiakela/deep-learning-research-and-practice/blob/main/deep-learning-with-pytorch-step-by-step/Part-III-NLP/06_tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Tokenization

We’ll start our NLP journey by following the steps of Alice and Dorothy, from
[Alice’s Adventures in Wonderland](https://ota.bodleian.ox.ac.uk/repository/xmlui/handle/20.500.12024/1476) by Lewis Carroll and [The Wonderful Wizard of Oz](https://ota.bodleian.ox.ac.uk/repository/xmlui/handle/20.500.12024/1740) by L. Frank Baum.

![](https://github.com/rahiakela/deep-learning-research-and-practice/blob/main/deep-learning-with-pytorch-step-by-step/Part-III-NLP/images/alice_dorothy.png?raw=1)

*Left: "Alice and the Baby Pig" illustration by John Tenniel's, from "Alice's Adventure's in Wonderland" (1865).*

*Right: "Dorothy meets the Cowardly Lion" illustration by W.W. Denslow, from "The Wonderful Wizard of Oz" (1900)*

##Setup

In [1]:
%%capture

# # UPDATED
# ###########################################################
!pip install gensim==4.3.1
# # The library has been archived and won't be used anymore
# # # !pip install allennlp==0.9.0
#!pip install flair==0.12.2
#!pip install torchvision==0.15.1
# # # HuggingFace
!pip install transformers==4.32.0
!pip install datasets==2.14.4
# ###########################################################
!pip -q install spacy
!python -m spacy download en_core_web_sm

In [2]:
try:
    import google.colab
    import requests
    url = 'https://raw.githubusercontent.com/dvgodoy/PyTorchStepByStep/master/config.py'
    r = requests.get(url, allow_redirects=True)
    open('config.py', 'wb').write(r.content)
except ModuleNotFoundError:
    pass

from config import *
config_chapter11()
# This is needed to render the plots in this chapter
from plots.chapter11 import *

Finished!


In [3]:
import os
import json
import errno
import requests
import numpy as np
from copy import deepcopy
from operator import itemgetter

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Dataset

from data_generation.nlp import ALICE_URL, WIZARD_URL, download_text
from stepbystep.v4 import StepByStep
# These are the classes we built in Chapter 10
from seq2seq import *

import spacy
import nltk
from nltk.tokenize import sent_tokenize

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
import gensim
from gensim import corpora, downloader
from gensim.parsing.preprocessing import *
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec

In [6]:
from datasets import load_dataset, Split
from transformers import (
    DataCollatorForLanguageModeling,
    BertModel, BertTokenizer, BertForSequenceClassification,
    DistilBertModel, DistilBertTokenizer,
    DistilBertForSequenceClassification,
    AutoModelForSequenceClassification,
    AutoModel, AutoTokenizer, AutoModelForCausalLM,
    Trainer, TrainingArguments, pipeline, TextClassificationPipeline
)
from transformers.pipelines import SUPPORTED_TASKS

##Downloading Books

In [7]:
!rm -rf data

In [8]:
# let's download data
HOME_DIR = "data"
download_text(ALICE_URL, HOME_DIR)
download_text(WIZARD_URL, HOME_DIR)

In [9]:
# let's see the downloaded data
#!cat data/alice28-1476.txt

In [10]:
#!cat data/wizoz10-1740.txt

We need to remove these additions to the original texts:

In [11]:
alice_file = os.path.join(HOME_DIR, "alice28-1476.txt")
with open(alice_file, "r") as f:
  # The actual texts of the books are contained between lines 105 and 3703
  alice_text = "".join(f.readlines()[104:3704])

wizard_file = os.path.join(HOME_DIR, "wizoz10-1740.txt")
with open(wizard_file, "r") as f:
  # The actual texts of the books are contained between lines 309 and 5099
  wizard_text = "".join(f.readlines()[310:5100])

In [12]:
print(alice_text[:500])
print("\n", "#"*70, "\n")
print(wizard_text[:500])

                ALICE'S ADVENTURES IN WONDERLAND

                          Lewis Carroll

               THE MILLENNIUM FULCRUM EDITION 2.8




                            CHAPTER I

                      Down the Rabbit-Hole


  Alice was beginning to get very tired of sitting by her sister
on the bank, and of having nothing to do:  once or twice she had
peeped into the book her sister was reading, but it had no
pictures or conversations in it, `and what is the use of a book,'
thought Alice `w

 ###################################################################### 

                    THE WONDERFUL WIZARD OF OZ


                          1.  The Cyclone


    Dorothy lived in the midst of the great Kansas prairies, with
Uncle Henry, who was a farmer, and Aunt Em, who was the farmer's
wife.  Their house was small, for the lumber to build it had to be
carried by wagon many miles.  There were four walls, a floor and a
roof, which made one room; and this room contained a rusty looking

We can partially automate the removal of the extra lines by setting the real start and end lines of each text in a configuration file.

In [13]:
text_cfg = """fname,start,end
alice28-1476.txt,104,3704
wizoz10-1740.txt,310,5100"""
bytes_written = open(os.path.join(HOME_DIR, 'lines.cfg'), 'w').write(text_cfg)

##Sentence Tokenization

A token is a piece of a text, and to tokenize a text means to split
it into pieces; that is, into a list of tokens.

The most common kind of piece is a word.

So, tokenizing a text usually means to
split it into words using the white space as a separator.

In [14]:
sentence = "I'm following the white rabbit"
tokens = sentence.split(" ")
tokens

["I'm", 'following', 'the', 'white', 'rabbit']

Let's do sentence tokenization, which means to split a text into its sentences.

In [15]:
corpus_alice = sent_tokenize(alice_text)
corpus_wizard = sent_tokenize(wizard_text)

len(corpus_alice), (len(corpus_wizard))

(1612, 2240)

Let’s check one sentence from the first corpus of text.

In [16]:
corpus_alice[2]

'There was nothing so VERY remarkable in that; nor did Alice\nthink it so VERY much out of the way to hear the Rabbit say to\nitself, `Oh dear!'

Let’s check one sentence from the second corpus of text.

In [17]:
corpus_wizard[30]

'"There\'s a cyclone coming, Em," he called to his wife.'

Our dataset is going to be a collection of CSV files, one file for each book, with each
CSV file containing one sentence per line.

Therefore, we need to:

* clean the line breaks to make sure each sentence is on one line only;
* define an appropriate quote char to "wrap" the sentence such that the original commas and semicolons in the original text do not get misinterpreted as separation chars of the CSV file; and
* add a second column to the CSV file to
identify the original source of the sentence since we’ll be concatenating, and
shuffling the sentences before training a model on our corpora.

The sentence above should end up looking like this:
```log
\"There's a cyclone coming, Em," he called to his wife.\,wizoz10-1740.txt
```

The function below does the grunt work of cleaning, splitting, and saving the
sentences to a CSV file for us:

In [18]:
def sentence_tokenize(source, quote_char="\\", sep_char=",", include_header=True, include_source=True, extensions=("txt"), **kwargs):
  # If source is a folder, goes through all files inside it that match the desired extensions ('txt' by default)
  if os.path.isdir(source):
    filenames = [f for f in os.listdir(source) if os.path.isfile(os.path.join(source, f)) and os.path.splitext(f)[1][1:] in extensions]
  elif isinstance(source, str):
    filenames = [source]

  # If there is a configuration file, builds a dictionary with the corresponding start and end lines of each text file
  config_file = os.path.join(source, "lines.cfg")
  config = {}
  if os.path.exists(config_file):
    with open(config_file, "r") as f:
      rows = f.readlines()
    for r in rows[1:]:
      fname, start, end = r.strip().split(",")
      config.update({fname: (int(start), int(end))})

  new_fnames = []
  # For each file of text
  for fname in filenames:
    # If there's a start and end line for that file, use it
    try:
        start, end = config[fname]
    except KeyError:
        start = None
        end = None

    # Opens the file, slices the configures lines (if any)
    # cleans line breaks and uses the sentence tokenizer
    with open(os.path.join(source, fname), 'r') as f:
        contents = (''.join(f.readlines()[slice(start, end, None)]).replace('\n', ' ').replace('\r', ''))
    corpus = sent_tokenize(contents, **kwargs)

    # Builds a CSV file containing tokenized sentences
    base = os.path.splitext(fname)[0]
    new_fname = f'{base}.sent.csv'
    new_fname = os.path.join(source, new_fname)
    with open(new_fname, 'w') as f:
        # Header of the file
        if include_header:
            if include_source:
                f.write('sentence,source\n')
            else:
                f.write('sentence\n')
        # Writes one line for each sentence
        for sentence in corpus:
            if include_source:
                f.write(f'{quote_char}{sentence}{quote_char}{sep_char}{fname}\n')
            else:
                f.write(f'{quote_char}{sentence}{quote_char}\n')
    new_fnames.append(new_fname)

  # Returns list of the newly generated CSV files
  return sorted(new_fnames)

In [19]:
new_fnames = sentence_tokenize(HOME_DIR)
new_fnames

['data/alice28-1476.sent.csv', 'data/wizoz10-1740.sent.csv']

##Spacy sentence tokenization

In [20]:
nlp = spacy.blank("en")
nlp.add_pipe("sentencizer")

sentences = []
for doc in nlp.pipe(corpus_alice):
  sentences.extend(sent.text for sent in doc.sents)

len(sentences), sentences[2]

(1615,
 'There was nothing so VERY remarkable in that; nor did Alice\nthink it so VERY much out of the way to hear the Rabbit say to\nitself, `Oh dear!')

##HuggingFace’s Dataset

In [21]:
# let's load from local files using HuggingFace
dataset = load_dataset(path="csv", data_files=new_fnames, quotechar="\\", split=Split.TRAIN)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [22]:
# let's see attributes, like features, num_columns, and shape
dataset.features, dataset.num_columns, dataset.shape

({'sentence': Value(dtype='string', id=None),
  'source': Value(dtype='string', id=None)},
 2,
 (3852, 2))

In [23]:
dataset[2]

{'sentence': 'There was nothing so VERY remarkable in that; nor did Alice think it so VERY much out of the way to hear the Rabbit say to itself, `Oh dear!',
 'source': 'alice28-1476.txt'}

In [24]:
dataset["sentence"][:3]

["                ALICE'S ADVENTURES IN WONDERLAND                            Lewis Carroll                 THE MILLENNIUM FULCRUM EDITION 2.8                                 CHAPTER I                        Down the Rabbit-Hole     Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do:  once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, `and what is the use of a book,' thought Alice `without pictures or conversation?'",
 'So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her.',
 'There was nothing so VERY remarkable in that; nor did Alice think it so VERY much out of the way to hear the Rabbit say to itself, `Oh dear!']

In [25]:
dataset["source"][:3]

['alice28-1476.txt', 'alice28-1476.txt', 'alice28-1476.txt']

In [26]:
# check the unique sources
dataset.unique("source")

['alice28-1476.txt', 'wizoz10-1740.txt']

In [27]:
# let's create new columns
def is_alice_label(row):
  is_alice = int(row["source"] == "alice28-1476.txt")
  return {"label": is_alice}

In [28]:
dataset = dataset.map(is_alice_label)

Map:   0%|          | 0/3852 [00:00<?, ? examples/s]

In [29]:
dataset[2]

{'sentence': 'There was nothing so VERY remarkable in that; nor did Alice think it so VERY much out of the way to hear the Rabbit say to itself, `Oh dear!',
 'source': 'alice28-1476.txt',
 'label': 1}

In [30]:
# Now, we can finally shuffle the dataset and split it into training and test sets
shuffled_dataset = dataset.shuffle(seed=42)
split_dataset = shuffled_dataset.train_test_split(test_size=0.2)
split_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'source', 'label'],
        num_rows: 3081
    })
    test: Dataset({
        features: ['sentence', 'source', 'label'],
        num_rows: 771
    })
})

In [31]:
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

In [32]:
train_dataset[0]

{'sentence': 'Even the grass was not green, for the sun had burned the tops of the long blades until they were the same gray color to be seen everywhere.',
 'source': 'wizoz10-1740.txt',
 'label': 0}

##Word Tokenization

In [33]:
# naïve word tokenization
sentence = "I'm following the white rabbit"
tokens = sentence.split(" ")
tokens

["I'm", 'following', 'the', 'white', 'rabbit']

In [34]:
# Let’s try using Gensim
preprocess_string(sentence)

['follow', 'white', 'rabbit']

In [35]:
# We won’t be removing stop words or performing stemming here
filters = [lambda x: x.lower(),
           strip_tags,
           strip_punctuation,
           strip_multiple_whitespaces,
           strip_numeric]

preprocess_string(sentence, filters=filters)

['i', 'm', 'following', 'the', 'white', 'rabbit']

In [36]:
# Another option
tokens = simple_preprocess(sentence)
tokens

['following', 'the', 'white', 'rabbit']

##Vocabulary

In [37]:
sentences = train_dataset["sentence"]
tokens = [simple_preprocess(sent) for sent in sentences]
tokens[0]

['even',
 'the',
 'grass',
 'was',
 'not',
 'green',
 'for',
 'the',
 'sun',
 'had',
 'burned',
 'the',
 'tops',
 'of',
 'the',
 'long',
 'blades',
 'until',
 'they',
 'were',
 'the',
 'same',
 'gray',
 'color',
 'to',
 'be',
 'seen',
 'everywhere']

In [38]:
# build a vocabulary
dictionary = corpora.Dictionary(tokens)
print(dictionary)

Dictionary<3719 unique tokens: ['be', 'blades', 'burned', 'color', 'even']...>


In [39]:
dictionary.num_docs

3081

In [40]:
dictionary.num_pos

51029

In [41]:
dictionary.token2id

{'be': 0,
 'blades': 1,
 'burned': 2,
 'color': 3,
 'even': 4,
 'everywhere': 5,
 'for': 6,
 'grass': 7,
 'gray': 8,
 'green': 9,
 'had': 10,
 'long': 11,
 'not': 12,
 'of': 13,
 'same': 14,
 'seen': 15,
 'sun': 16,
 'the': 17,
 'they': 18,
 'to': 19,
 'tops': 20,
 'until': 21,
 'was': 22,
 'were': 23,
 'and': 24,
 'bade': 25,
 'buttercups': 26,
 'bye': 27,
 'daisies': 28,
 'dotted': 29,
 'fields': 30,
 'good': 31,
 'here': 32,
 'him': 33,
 'over': 34,
 'soft': 35,
 'thanked': 36,
 'there': 37,
 'toward': 38,
 'turned': 39,
 'walking': 40,
 'west': 41,
 'with': 42,
 'cried': 43,
 'she': 44,
 'all': 45,
 'carry': 46,
 'command': 47,
 'do': 48,
 'go': 49,
 'have': 50,
 'heels': 51,
 'is': 52,
 'knock': 53,
 'shoes': 54,
 'three': 55,
 'times': 56,
 'together': 57,
 'wherever': 58,
 'wish': 59,
 'you': 60,
 'asked': 61,
 'child': 62,
 'began': 63,
 'came': 64,
 'dorothy': 65,
 'evening': 66,
 'her': 67,
 'house': 68,
 'larger': 69,
 'night': 70,
 'pass': 71,
 'rather': 72,
 'rest': 73,
 '

In [42]:
vocab = list(dictionary.token2id.keys())
vocab[:5]

['be', 'blades', 'burned', 'color', 'even']

In [43]:
# collection frequencies
dictionary.cfs

{4: 40,
 17: 3678,
 7: 18,
 22: 688,
 12: 295,
 9: 88,
 6: 380,
 16: 22,
 10: 341,
 2: 3,
 20: 3,
 13: 1104,
 11: 78,
 1: 2,
 21: 56,
 18: 447,
 23: 234,
 14: 32,
 8: 11,
 3: 12,
 19: 1486,
 0: 260,
 15: 24,
 5: 6,
 36: 6,
 33: 177,
 24: 2008,
 25: 7,
 31: 102,
 27: 21,
 39: 22,
 38: 15,
 41: 20,
 40: 20,
 34: 101,
 30: 7,
 35: 8,
 29: 2,
 32: 75,
 37: 171,
 42: 358,
 28: 3,
 26: 2,
 44: 754,
 43: 40,
 45: 334,
 60: 741,
 50: 191,
 48: 175,
 52: 295,
 53: 2,
 51: 3,
 57: 29,
 55: 47,
 56: 18,
 47: 10,
 54: 28,
 46: 26,
 58: 4,
 59: 43,
 49: 99,
 61: 104,
 62: 22,
 66: 7,
 78: 190,
 65: 304,
 76: 13,
 67: 528,
 77: 17,
 63: 73,
 80: 21,
 79: 82,
 74: 63,
 71: 13,
 70: 23,
 64: 113,
 68: 39,
 72: 32,
 69: 8,
 75: 62,
 73: 17,
 95: 31,
 90: 184,
 89: 206,
 94: 27,
 92: 1,
 86: 814,
 88: 208,
 84: 82,
 83: 17,
 87: 151,
 85: 456,
 91: 630,
 97: 158,
 81: 6,
 82: 31,
 93: 2,
 96: 259,
 102: 367,
 99: 34,
 103: 2,
 98: 103,
 101: 45,
 100: 35,
 104: 120,
 106: 8,
 112: 27,
 113: 109,
 108: 4

In [44]:
# document frequencies
dictionary.dfs

{4: 39,
 17: 1885,
 7: 18,
 22: 547,
 12: 267,
 9: 69,
 6: 351,
 16: 22,
 10: 300,
 2: 3,
 20: 3,
 13: 827,
 11: 76,
 1: 2,
 21: 56,
 18: 344,
 23: 204,
 14: 32,
 8: 11,
 3: 11,
 19: 1091,
 0: 239,
 15: 24,
 5: 6,
 36: 6,
 33: 152,
 24: 1305,
 25: 7,
 31: 95,
 27: 21,
 39: 22,
 38: 15,
 41: 18,
 40: 20,
 34: 99,
 30: 7,
 35: 8,
 29: 2,
 32: 74,
 37: 164,
 42: 315,
 28: 3,
 26: 2,
 44: 509,
 43: 40,
 45: 307,
 60: 571,
 50: 176,
 48: 171,
 52: 263,
 53: 2,
 51: 3,
 57: 28,
 55: 46,
 56: 18,
 47: 10,
 54: 28,
 46: 26,
 58: 4,
 59: 43,
 49: 97,
 61: 104,
 62: 22,
 66: 7,
 78: 180,
 65: 299,
 76: 13,
 67: 392,
 77: 17,
 63: 72,
 80: 21,
 79: 82,
 74: 61,
 71: 13,
 70: 23,
 64: 110,
 68: 38,
 72: 32,
 69: 8,
 75: 60,
 73: 17,
 95: 30,
 90: 180,
 89: 181,
 94: 27,
 92: 1,
 86: 607,
 88: 196,
 84: 81,
 83: 17,
 87: 149,
 85: 358,
 91: 624,
 97: 156,
 81: 5,
 82: 31,
 93: 2,
 96: 227,
 102: 352,
 99: 34,
 103: 2,
 98: 102,
 101: 39,
 100: 35,
 104: 114,
 106: 6,
 112: 25,
 113: 106,
 108: 46,


In [45]:
# doc to id
sentence = "follow the white rabbit"
new_tokens = simple_preprocess(sentence)
ids = dictionary.doc2idx(new_tokens)
print(new_tokens)
print(ids)

['follow', 'the', 'white', 'rabbit']
[2276, 17, 507, 721]


In [46]:
# add special token to unknown word
special_tokens = {
    "[PAD]": 0,
    "[UNK]": 1
}

dictionary.patch_with_special_tokens(special_tokens)

In [47]:
def get_rare_ids(dictionary, min_freq):
  rare_ids = [t[0] for t in dictionary.cfs.items() if t[1] < min_freq]
  return rare_ids

In [48]:
def make_vocab(sentences, folder=None, special_tokens=None, vocab_size=None, min_freq=None):

  if folder is not None:
    if not os.path.exists(folder):
      os.mkdir(folder)

  # tokenizes the sentences and create a Dictionary
  tokens = [simple_preprocess(sent) for sent in sentences]
  dictionary = corpora.Dictionary(tokens)

  # keeps only the most frequent words (vocab size)
  if vocab_size is not None:
    dictionary.filter_extremes(keep_n=vocab_size)

  # removes rare words (in case the vocab size still includes words with low frequency)
  if min_freq is not None:
    rare_tokens = get_rare_ids(dictionary, min_freq)
    dictionary.filter_tokens(bad_ids=rare_tokens)

  # gets the whole list of tokens and frequencies
  items = dictionary.cfs.items()

  # sorts the tokens in descending order
  words = [dictionary[t[0]] for t in sorted(dictionary.cfs.items(), key=lambda t: -t[1])]

  # prepends special tokens, if any
  if special_tokens is not None:
    to_add = []
    for special_token in special_tokens:
      if special_token not in words:
        to_add.append(special_token)
    words = to_add + words

  with open(os.path.join(folder, "vocab.txt"), "w") as f:
    for word in words:
      f.write(f"{word}\n")

In [49]:
make_vocab(
    train_dataset["sentence"],
    "our_vocab/",
    special_tokens=['[PAD]', '[UNK]', '[SEP]', '[CLS]', '[MASK]'],
    min_freq=2
)

## HugginFace's Tokenizer

So, before loading a pre-trained tokenizer, let’s create our own tokenizer using our
own vocabulary.

In [50]:
tokenizer = BertTokenizer("our_vocab/vocab.txt")

In [51]:
# Then, let’s tokenize a new sentence
new_sentence = "follow the white rabbit neo"
new_tokens = tokenizer.tokenize(new_sentence)
new_tokens

['follow', 'the', 'white', 'rabbit', '[UNK]']

In [52]:
# get the indices
new_ids = tokenizer.convert_tokens_to_ids(new_tokens)
new_ids

[965, 5, 205, 198, 1]

In [53]:
# perform two steps at once
new_ids = tokenizer.encode(new_sentence)
new_ids

[3, 965, 5, 205, 198, 1, 2]

In [54]:
# check the extra tokens
tokenizer.convert_ids_to_tokens(new_ids)

['[CLS]', 'follow', 'the', 'white', 'rabbit', '[UNK]', '[SEP]']

In [55]:
# get rid of the special tokens
tokenizer.encode(new_sentence, add_special_tokens=False)

[965, 5, 205, 198, 1]

In [56]:
# we can simply call the tokenizer itself
tokenizer(new_sentence, add_special_tokens=False, return_tensors="pt")

{'input_ids': tensor([[965,   5, 205, 198,   1]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [59]:
# token_type_ids only makes sense if the input has more than one sentence
sentence1 = "follow the white rabbit neo"
sentence2 = "no one can be told what the matrix is"
joined_sentences = tokenizer(sentence1, sentence2)
joined_sentences

{'input_ids': [3, 965, 5, 205, 198, 1, 2, 52, 44, 70, 32, 261, 43, 5, 1, 30, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [60]:
# Let’s convert the IDs back to tokens and inspect the result
print(tokenizer.convert_ids_to_tokens(joined_sentences["input_ids"]))

['[CLS]', 'follow', 'the', 'white', 'rabbit', '[UNK]', '[SEP]', 'no', 'one', 'can', 'be', 'told', 'what', 'the', '[UNK]', 'is', '[SEP]']


In [61]:
# we may pad the sequences to get them all with the same length
separate_sentences = tokenizer([sentence1, sentence2], padding=True)
separate_sentences

{'input_ids': [[3, 965, 5, 205, 198, 1, 2, 0, 0, 0, 0], [3, 52, 44, 70, 32, 261, 43, 5, 1, 30, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [62]:
# Let’s convert the IDs back to tokens again
print(tokenizer.convert_ids_to_tokens(separate_sentences["input_ids"][0]))
print(separate_sentences["attention_mask"][0])

['[CLS]', 'follow', 'the', 'white', 'rabbit', '[UNK]', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
